In [1]:
%run -i 'botTrain.py'

[nltk_data] Downloading package punkt to /home/head/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/head/Рабочий стол/Младший/Docker/rsvRUbot/rsvbot/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Части [100/1000], Успехи: 0.0000
Части [200/1000], Успехи: 0.0000
Части [300/1000], Успехи: 0.0000
Части [400/1000], Успехи: 0.0000
Части [500/1000], Успехи: 0.0000
Части [600/1000], Успехи: 0.0000
Части [700/1000], Успехи: 0.0000
Части [800/1000], Успехи: 0.0000
Части [900/1000], Успехи: 0.0000
Части [1000/1000], Успехи: 0.0000
Финальный шанс: 0.0000
training complete. file saved to botContent/skills/textskill.pth


In [2]:
import random
import json

import torch

from botModel import NeuralNet
from botUtils import wdBag, token

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('botContent/data/skilld.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "botContent/skills/textskill.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=10, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
)

In [3]:
bot_name = "RSVBot"
print("Добро пожаловать! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("Я: ")
    query = f"Я: {sentence}"
    if sentence == "quit":
        break

    sentence = token(sentence)
    X = wdBag(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(query)
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Извините, я не понял запрос...")

Добро пожаловать! (type 'quit' to exit)
Я: ['Привет']
RSVBot: Привет :-)
Я: ['Как', 'дела']
RSVBot: Какие свои возможности могу тебе показать?
